In [1]:
# Model 1: Random Forest Regressor
import pandas as pd
import numpy as np
import re

import sklearn
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

In [2]:
from xgboost import XGBRegressor, XGBRFRegressor
from catboost import CatBoostRegressor
import catboost
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge, Lasso, LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_transformer, ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.neural_network import MLPRegressor

from scipy.stats.mstats import winsorize

from pyod.models.feature_bagging import FeatureBagging
from pyod.models.lof import LOF

In [3]:
# Model 1: Random Forest Regressor

In [4]:
#CSV to Load
zillow_data= 'zillow_date_column.csv'
zillow_df=pd.read_csv(zillow_data, index_col=0, parse_dates=True,squeeze=True)
zillow_df.replace(np.nan,999.99)
zillow_df.head(100)

,87120,87121,87124,87109,87108,87110,87144,87106,87102,18103,...,22044,22181,20166,20036,67203,27103,27104,27012,27101,1604
Jan-14,791.0,1010.0,943.0,856.0,500.0,1173.0,1215.0,796.0,627.0,NaN,...,NaN,NaN,1781.0,NaN,654.0,996.0,NaN,NaN,NaN,1090.0
Feb-14,823.0,989.0,913.0,893.0,544.0,1152.0,1195.0,845.0,688.0,901.0,...,1424.0,2304.0,NaN,NaN,611.0,1050.0,940.0,1093.0,1064.0,1087.0
Mar-14,808.0,966.0,916.0,882.0,531.0,1170.0,1198.0,835.0,NaN,850.0,...,NaN,2182.0,1832.0,2232.0,NaN,1037.0,940.0,NaN,1030.0,1096.0
Apr-14,821.0,985.0,931.0,878.0,534.0,1172.0,1195.0,859.0,660.0,858.0,...,1431.0,2129.0,1815.0,NaN,NaN,1036.0,966.0,1095.0,1035.0,1089.0
May-14,832.0,996.0,948.0,871.0,523.0,1179.0,1209.0,880.0,676.0,NaN,...,1420.0,2103.0,1819.0,2164.0,707.0,1052.0,982.0,NaN,1050.0,1098.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
May-21,1111.0,1320.0,1294.0,1184.0,771.0,1543.0,1616.0,1250.0,962.0,1425.0,...,1576.0,2350.0,2011.0,2099.0,836.0,1441.0,1350.0,1449.0,1507.0,1723.0
Jun-21,1142.0,1364.0,1320.0,1214.0,765.0,1526.0,1665.0,1270.0,977.0,1462.0,...,1630.0,2369.0,2008.0,2107.0,856.0,1470.0,1346.0,1520.0,1663.0,1716.0
Jul-21,1177.0,1482.0,1336.0,1282.0,766.0,1486.0,1688.0,1324.0,974.0,1460.0,...,1668.0,2360.0,2034.0,2119.0,840.0,1520.0,1397.0,1532.0,1637.0,1732.0
City,Albuquerque,Albuquerque,Albuquerque,Albuquerque,Albuquerque,Albuquerque,Albuquerque,Albuquerque,Albuquerque,Allentown,...,Washington,Washington,Washington,Washington,Wichita,Winston-Salem,Winston-Salem,Winston-Salem,Winston-Salem,Worcester


In [6]:
zillow_df1 = zillow_df.replace(np.nan,999.99)

In [8]:
zillow_df1.tail(100)

,87120,87121,87124,87109,87108,87110,87144,87106,87102,18103,...,22044,22181,20166,20036,67203,27103,27104,27012,27101,1604
Jan-14,791.0,1010.0,943.0,856.0,500.0,1173.0,1215.0,796.0,627.0,999.99,...,999.99,999.99,1781.0,999.99,654.0,996.0,999.99,999.99,999.99,1090.0
Feb-14,823.0,989.0,913.0,893.0,544.0,1152.0,1195.0,845.0,688.0,901.0,...,1424.0,2304.0,999.99,999.99,611.0,1050.0,940.0,1093.0,1064.0,1087.0
Mar-14,808.0,966.0,916.0,882.0,531.0,1170.0,1198.0,835.0,999.99,850.0,...,999.99,2182.0,1832.0,2232.0,999.99,1037.0,940.0,999.99,1030.0,1096.0
Apr-14,821.0,985.0,931.0,878.0,534.0,1172.0,1195.0,859.0,660.0,858.0,...,1431.0,2129.0,1815.0,999.99,999.99,1036.0,966.0,1095.0,1035.0,1089.0
May-14,832.0,996.0,948.0,871.0,523.0,1179.0,1209.0,880.0,676.0,999.99,...,1420.0,2103.0,1819.0,2164.0,707.0,1052.0,982.0,999.99,1050.0,1098.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
May-21,1111.0,1320.0,1294.0,1184.0,771.0,1543.0,1616.0,1250.0,962.0,1425.0,...,1576.0,2350.0,2011.0,2099.0,836.0,1441.0,1350.0,1449.0,1507.0,1723.0
Jun-21,1142.0,1364.0,1320.0,1214.0,765.0,1526.0,1665.0,1270.0,977.0,1462.0,...,1630.0,2369.0,2008.0,2107.0,856.0,1470.0,1346.0,1520.0,1663.0,1716.0
Jul-21,1177.0,1482.0,1336.0,1282.0,766.0,1486.0,1688.0,1324.0,974.0,1460.0,...,1668.0,2360.0,2034.0,2119.0,840.0,1520.0,1397.0,1532.0,1637.0,1732.0
City,Albuquerque,Albuquerque,Albuquerque,Albuquerque,Albuquerque,Albuquerque,Albuquerque,Albuquerque,Albuquerque,Allentown,...,Washington,Washington,Washington,Washington,Wichita,Winston-Salem,Winston-Salem,Winston-Salem,Winston-Salem,Worcester


In [16]:
# delete a city & state rows at index values 91 & 92.
zillow_df2 = zillow_df1.drop(labels=['City', 'State'], axis=0)
# Rename Date column from "Unnamed:0 to Date"
#zillow_df1.rename(columns={'Unnamed: 0': 'Date'}, inplace=True)


In [17]:
zillow_df2.head(100)

,87120,87121,87124,87109,87108,87110,87144,87106,87102,18103,...,22044,22181,20166,20036,67203,27103,27104,27012,27101,1604
Jan-14,791.0,1010.0,943.0,856.0,500.0,1173.0,1215.0,796.0,627.0,999.99,...,999.99,999.99,1781.0,999.99,654.0,996.0,999.99,999.99,999.99,1090.0
Feb-14,823.0,989.0,913.0,893.0,544.0,1152.0,1195.0,845.0,688.0,901.0,...,1424.0,2304.0,999.99,999.99,611.0,1050.0,940.0,1093.0,1064.0,1087.0
Mar-14,808.0,966.0,916.0,882.0,531.0,1170.0,1198.0,835.0,999.99,850.0,...,999.99,2182.0,1832.0,2232.0,999.99,1037.0,940.0,999.99,1030.0,1096.0
Apr-14,821.0,985.0,931.0,878.0,534.0,1172.0,1195.0,859.0,660.0,858.0,...,1431.0,2129.0,1815.0,999.99,999.99,1036.0,966.0,1095.0,1035.0,1089.0
May-14,832.0,996.0,948.0,871.0,523.0,1179.0,1209.0,880.0,676.0,999.99,...,1420.0,2103.0,1819.0,2164.0,707.0,1052.0,982.0,999.99,1050.0,1098.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Mar-21,1068.0,1300.0,1242.0,1140.0,735.0,1482.0,1557.0,1216.0,810.0,1368.0,...,1556.0,2304.0,1969.0,2089.0,846.0,1399.0,1259.0,1456.0,1444.0,1680.0
Apr-21,1093.0,1300.0,1260.0,1164.0,763.0,1508.0,1583.0,1242.0,844.0,1421.0,...,1557.0,2329.0,2007.0,2111.0,854.0,1433.0,1345.0,1455.0,1469.0,1671.0
May-21,1111.0,1320.0,1294.0,1184.0,771.0,1543.0,1616.0,1250.0,962.0,1425.0,...,1576.0,2350.0,2011.0,2099.0,836.0,1441.0,1350.0,1449.0,1507.0,1723.0
Jun-21,1142.0,1364.0,1320.0,1214.0,765.0,1526.0,1665.0,1270.0,977.0,1462.0,...,1630.0,2369.0,2008.0,2107.0,856.0,1470.0,1346.0,1520.0,1663.0,1716.0


In [18]:
#Index to Date Time Format
zillow_df2.index = pd.to_datetime(zillow_df2.index, format="%b-%y")

In [19]:
zillow_df2.head(100)

,87120,87121,87124,87109,87108,87110,87144,87106,87102,18103,...,22044,22181,20166,20036,67203,27103,27104,27012,27101,1604
2014-01-01,791.0,1010.0,943.0,856.0,500.0,1173.0,1215.0,796.0,627.0,999.99,...,999.99,999.99,1781.0,999.99,654.0,996.0,999.99,999.99,999.99,1090.0
2014-02-01,823.0,989.0,913.0,893.0,544.0,1152.0,1195.0,845.0,688.0,901.0,...,1424.0,2304.0,999.99,999.99,611.0,1050.0,940.0,1093.0,1064.0,1087.0
2014-03-01,808.0,966.0,916.0,882.0,531.0,1170.0,1198.0,835.0,999.99,850.0,...,999.99,2182.0,1832.0,2232.0,999.99,1037.0,940.0,999.99,1030.0,1096.0
2014-04-01,821.0,985.0,931.0,878.0,534.0,1172.0,1195.0,859.0,660.0,858.0,...,1431.0,2129.0,1815.0,999.99,999.99,1036.0,966.0,1095.0,1035.0,1089.0
2014-05-01,832.0,996.0,948.0,871.0,523.0,1179.0,1209.0,880.0,676.0,999.99,...,1420.0,2103.0,1819.0,2164.0,707.0,1052.0,982.0,999.99,1050.0,1098.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-03-01,1068.0,1300.0,1242.0,1140.0,735.0,1482.0,1557.0,1216.0,810.0,1368.0,...,1556.0,2304.0,1969.0,2089.0,846.0,1399.0,1259.0,1456.0,1444.0,1680.0
2021-04-01,1093.0,1300.0,1260.0,1164.0,763.0,1508.0,1583.0,1242.0,844.0,1421.0,...,1557.0,2329.0,2007.0,2111.0,854.0,1433.0,1345.0,1455.0,1469.0,1671.0
2021-05-01,1111.0,1320.0,1294.0,1184.0,771.0,1543.0,1616.0,1250.0,962.0,1425.0,...,1576.0,2350.0,2011.0,2099.0,836.0,1441.0,1350.0,1449.0,1507.0,1723.0
2021-06-01,1142.0,1364.0,1320.0,1214.0,765.0,1526.0,1665.0,1270.0,977.0,1462.0,...,1630.0,2369.0,2008.0,2107.0,856.0,1470.0,1346.0,1520.0,1663.0,1716.0


In [20]:
zillow_df2.to_csv('zillow_df2.csv')

In [ ]:
zillow_df1.head(100)

In [ ]:
zillow_df = zillow_df.drop(['state','metro_code', 'fips2010','areaname','county_town_name','metro'], axis=1)
FMR22_studio_df.head()

In [ ]:
#CLEANING DATA--WHAT NEEDS TO BE DONE:
#Delete “RegionType” as all are state
#Rename “RegionName” to “State”
#Delete “StateName” as its a redundancy, 

#Each column with the avg home values we will have to rename each column accordingly,
#  i.e.: “1996-01-31” gets renamed to something like “HV_1_1996" (Home Value for January 1996") 
#  Considering its every month from 1996 to 2021, thats….(doing math…) 300 columns…so 
#  it’ll be a bit of work renaming we can all jump in on and split, but that way we 
#  have very meaningful, self-explanatory, organized data

##Not sure what to do about RegionID and SizeRank… 
##  we will have to figure out if we need them

In [ ]:
#Import from SKLearn Models -GridSearchCV & RandomForestRegressor

In [ ]:
#Split Training Data (X_Train & Y_Train)

In [ ]:
#Set Grid Parameters & Initialise both models

In [ ]:
#Fit & Train Models

In [ ]:
#Record Results

In [ ]:
#Create Visual Representation 